In [ ]:
%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
%load_ext tensorboard

# !sudo apt-get install -y xvfb ffmpeg
# !pip install 'xvfbwrapper==0.2.9'
# !pip install 'gym==0.10.11'
# !pip install 'imageio==2.4.0'
# !pip install PILLOW
# !pip install 'pyglet==1.3.2'
# !pip install pyvirtualdisplay
# !pip install tf-agents
# !pip install gast

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
import os
from datetime import datetime
import tensorflow as tf
import sklearn

import abc
import base64
import imageio
import io
import IPython
import numpy as np
import PIL.Image
import pyvirtualdisplay
import tensorflow_probability as tfp
import numpy as np
import shutil
import tempfile
import zipfile

pd.set_option('mode.chained_assignment', None)
sn.set(rc={'figure.figsize':(9,9)})
sn.set(font_scale=1.4)

# make results reproducible
seed = 0
np.random.seed(seed)
tf.random.set_seed(13)

# Value and Policy based methods

So far we have looked at value based methods, or approaches to reinforcement learning, such as DQN methods. Let us now look at policy based methods, such as PPO.


# Value Based Methods

Learn a harder problem, epilson greedy exploration

* Vanilla DQN `DqnAgent`
* PDD DQN `DdpgAgent`
* Double Q-Learning ?
* SARSA
* Value-iteration ?

# Policy Based Methods 

Learn an easier problem

* Reinforce here ! 
* A2C ``
* PPO `PPOAgent`

# Off-policy Policy Based Methods

* SAC `SacAgent`
* SIL (not with A2C, PPO but SAC)

# Exploration Techniques 

* Thompson sampling with MCDO
* RND

#  Uncertainty in RL

* Categorical DQN(C51) `CategoricalDqnAgent`
* QR-DQN
* Implicit Quantile Networks

#  Imitation Learning 
* GAIL

# Multi-Agent RL
* Upper Confidence Bounds for Tree(UCT)
* Counterfactual Hedge


**Main Networks**

* **QNetwork**: Used in Qlearning for environments with discrete actions, this network maps an observation to value estimates for each possible action.
* **CriticNetworks**: Also referred to as `ValueNetworks` in literature, learns to estimate some version of a Value function mapping some state into an estimate for the expected return of a policy. These networks estimate how good the state the agent is currently in is.
* **ActorNetworks**: Learn a mapping from observations to actions. These networks are usually used by our policies to generate actions.
* **ActorDistributionNetworks**: Similar to `ActorNetworks` but these generate a distribution which a policy can then sample to generate actions.

In [ ]:
from tf_agents.agents.behavioral_cloning.behavioral_cloning_agent import BehavioralCloningAgent
from tf_agents.agents.categorical_dqn.categorical_dqn_agent import CategoricalDqnAgent
from tf_agents.agents.ddpg.ddpg_agent import DdpgAgent
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.agents.ppo.ppo_agent import PPOAgent
from tf_agents.agents.reinforce.reinforce_agent import ReinforceAgent
from tf_agents.agents.sac.sac_agent import SacAgent
from tf_agents.agents.td3.td3_agent import Td3Agent
